In [1]:
import numpy as np
import pandas as pd

In [2]:
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
df = pd.read_csv('u.data', sep='\t', names=column_names)

In [3]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


Nos falta el nombre de la película....

In [4]:
movie_titles = pd.read_csv("Movie_Id_Titles")
movie_titles.head()

,item_id,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


Se unen las bases a traves de un merge....

In [5]:
df = pd.merge(df,movie_titles,on='item_id')
df.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


Veamos cuantos usuarios y películas únicas tenemos...

In [6]:
n_users = df.user_id.nunique()
n_items = df.item_id.nunique()

print('Num. of Users: '+ str(n_users))
print('Num of Movies: '+str(n_items))

Num. of Users: 943
Num of Movies: 1682


## Data Parition

Debemos hacer el data parttion con el que vamos a trabajar, en este caso vamos a usar el 80% para entreamiento y 20% para test.

El objetivo es tener datos para probar las predicciones de calificaciones  a través del SVD.

In [7]:
from sklearn.cross_validation import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2)

/home/abraham/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Ejemplo de user-item matrix:
<img class="aligncenter size-thumbnail img-responsive" src="BLOG_CCA_8.png" alt="blog8"/>

Ahora debemos convertir los datos en una matriz que contenga los usuarios como renglones y las columnas como peliculas, como se mencionó al principio...

In [8]:
#Create two user-item matrices, one for training and another for testing
train_data_matrix = np.zeros((n_users, n_items))
for line in train_data.itertuples():
    train_data_matrix[line[1]-1, line[2]-1] = line[3]  

test_data_matrix = np.zeros((n_users, n_items))
for line in test_data.itertuples():
    test_data_matrix[line[1]-1, line[2]-1] = line[3]

Ahora tenemos la matriz y ya podemos aplicar SVD...

In [9]:
train_data_matrix

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

### SVD
A well-known matrix factorization method is **Singular value decomposition (SVD)**. Collaborative Filtering can be formulated by approximating a matrix `X` by using singular value decomposition. The winning team at the Netflix Prize competition used SVD matrix factorization models to produce product recommendations, for more information I recommend to read articles: [Netflix Recommendations: Beyond the 5 stars](http://techblog.netflix.com/2012/04/netflix-recommendations-beyond-5-stars.html) and [Netflix Prize and SVD](http://buzzard.ups.edu/courses/2014spring/420projects/math420-UPS-spring-2014-gower-netflix-SVD.pdf).
The general equation can be expressed as follows:
<img src="https://latex.codecogs.com/gif.latex?X=USV^T" title="X=USV^T" />


Given `m x n` matrix `X`:
* *`U`* is an *`(m x r)`* orthogonal matrix
* *`S`* is an *`(r x r)`* diagonal matrix with non-negative real numbers on the diagonal
* *V^T* is an *`(r x n)`* orthogonal matrix

Elements on the diagnoal in `S` are known as *singular values of `X`*. 


Matrix *`X`* can be factorized to *`U`*, *`S`* and *`V`*. The *`U`* matrix represents the feature vectors corresponding to the users in the hidden feature space and the *`V`* matrix represents the feature vectors corresponding to the items in the hidden feature space.
<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="BLOG_CCA_4.png"/>

Now you can make a prediction by taking dot product of *`U`*, *`S`* and *`V^T`*.

<img class="aligncenter size-thumbnail img-responsive" style="max-width:100%; width: 50%; max-width: none" src="BLOG_CCA_5.png"/>

Definimos root mean suared error como métrica para evaluar predicciones de ratings...

In [10]:
from sklearn.metrics import mean_squared_error
from math import sqrt
def rmse(prediction, ground_truth):
    prediction = prediction[ground_truth.nonzero()].flatten() 
    ground_truth = ground_truth[ground_truth.nonzero()].flatten()
    return sqrt(mean_squared_error(prediction, ground_truth))

Hacemos SVD com 10 factores usando scipy...

In [69]:
import scipy.sparse as sp
from scipy.sparse.linalg import svds

#get SVD components from train matrix. Choose k.
u, s, vt = svds(train_data_matrix,k=90)
s_diag_matrix=np.diag(s)
X_pred = np.dot(np.dot(u, s_diag_matrix), vt)


In [70]:
print('User-based CF MSE: ' + str(rmse(X_pred, test_data_matrix)))

User-based CF MSE: 3.125898090147294


Carelessly addressing only the relatively few known entries is highly prone to overfitting. SVD can be very slow and computationally expensive. More recent work minimizes the squared error by applying alternating least square or stochastic gradient descent and uses regularization terms to prevent overfitting. Alternating least square and stochastic gradient descent methods for CF will be covered in the next tutorials.


In [13]:
X_pred

array([[  3.36925323e+00,   1.80692294e+00,   9.26358664e-01, ...,
         -7.18297387e-03,   2.76884585e-02,   0.00000000e+00],
       [  1.57973152e+00,  -4.51489014e-02,   1.28513494e-01, ...,
          1.02554245e-02,   1.28414505e-03,   0.00000000e+00],
       [ -1.25916557e-01,  -7.64149756e-02,   7.14835915e-02, ...,
          1.79939758e-02,  -2.48033607e-03,   0.00000000e+00],
       ..., 
       [  1.54618006e+00,  -8.55687540e-02,   2.12877678e-01, ...,
         -1.66078411e-03,  -3.90398342e-03,   0.00000000e+00],
       [  1.40082389e+00,   5.54048392e-02,  -4.64107376e-01, ...,
          1.59499597e-02,  -2.49175881e-03,   0.00000000e+00],
       [  1.49485234e+00,   1.58047074e+00,   7.57591317e-01, ...,
         -6.20756724e-03,   1.93855187e-02,   0.00000000e+00]])

In [14]:
train_data_matrix

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In [15]:
np.shape(X_pred)

(943, 1682)

In [16]:
np.savetxt('train_data_matrix.txt', train_data_matrix)

In [18]:
np.shape(train_data_matrix)

(943, 1682)

In [20]:
train_data_matrix.transpose()

array([[ 5.,  0.,  0., ...,  5.,  0.,  0.],
       [ 3.,  0.,  0., ...,  0.,  0.,  5.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [139]:
s-ss

array([  1.49200429e+02,   5.56027401e+01,   7.38396217e+01,
         4.31925442e+01,   4.61925604e+01,   3.75036448e+01,
         2.21545887e+01,   2.12125427e+01,   1.01027961e+01,
         6.09398969e+00,   3.14653226e+00,   3.19970249e+00,
         2.02260798e+00,   1.82475985e-01,  -6.02883008e-01,
        -2.63430452e+00,  -3.81853403e+00,  -3.75052941e+00,
        -5.08066984e+00,  -4.86628612e+00,  -4.55930951e+00,
        -5.09533751e+00,  -5.60346959e+00,  -5.72461194e+00,
        -5.58716985e+00,  -5.70761373e+00,  -5.92608087e+00,
        -6.12752967e+00,  -6.08856489e+00,  -6.39872342e+00,
        -6.53315335e+00,  -7.02219752e+00,  -6.67025448e+00,
        -6.88879112e+00,  -7.08538677e+00,  -6.93656737e+00,
        -6.58901656e+00,  -6.89796376e+00,  -7.41996269e+00,
        -7.38433604e+00,  -7.45522176e+00,  -7.06464177e+00,
        -7.03265811e+00,  -6.91470260e+00,  -7.05679903e+00,
        -7.04156247e+00,  -6.96728243e+00,  -7.09962942e+00,
        -7.05455560e+00,

In [334]:
%%time 
from numpy import *
U,s,V = linalg.svd(train_data_matrix.transpose())

CPU times: user 4.74 s, sys: 2.39 s, total: 7.13 s
Wall time: 933 ms


In [ ]:
user 1.55s system 99% cpu 6.596 total 5.04s 

In [132]:
np.shape(V)

(943, 943)

In [163]:
pred=np.dot(np.dot(U[:,:943],np.diag(s[:943])),V[0:943,:])
prt=pred.transpose()

In [164]:
np.shape(prt)

(943, 1682)

In [165]:
print('User-based CF MSE: ' + str(rmse(prt, test_data_matrix)))

User-based CF MSE: 3.70020269715


In [167]:
ss=np.loadtxt('S.txt')

In [168]:
U=np.loadtxt('U .txt')

In [169]:
np.shape(U)

(1682, 1682)

In [170]:
VT=np.loadtxt('VT .txt')

In [171]:
np.shape(VT)

(943, 943)

In [172]:
predT=np.matrix(U[:, :943]) * np.diag(ss[:943]) * np.matrix(VT[:943, :])

In [173]:
predcuda=predT.transpose()

In [118]:
np.shape(predcuda)

(943, 1682)

In [119]:
print('User-based CF MSE: ' + str(rmse(np.array(predcuda), test_data_matrix)))

User-based CF MSE: 3.54497401718


In [121]:
pred

array([[  5.00000000e+00,   3.00000000e+00,   4.00000000e+00, ...,
         -2.61075883e-16,   1.24683250e-16,   0.00000000e+00],
       [ -9.98871125e-14,   1.55847557e-13,  -2.87547763e-14, ...,
          5.11743425e-17,   3.46944695e-18,   0.00000000e+00],
       [  2.29087582e-14,  -1.01486528e-13,  -4.19247970e-14, ...,
         -9.02056208e-17,  -2.02962647e-16,   0.00000000e+00],
       ..., 
       [  5.00000000e+00,   2.90566182e-16,   1.34614542e-15, ...,
         -5.48172618e-16,  -1.57859836e-16,   0.00000000e+00],
       [ -1.07006418e-14,  -1.69794734e-14,  -6.34908792e-16, ...,
          4.16333634e-17,  -1.76941795e-16,   0.00000000e+00],
       [ -2.10075013e-14,   5.00000000e+00,  -4.59007832e-15, ...,
          2.47198095e-16,   5.10008702e-16,   0.00000000e+00]])

In [120]:
predcuda

matrix([[  1.01566779e+00,   1.90207861e+00,   8.50892734e-01, ...,
           1.99125770e-02,   9.38555380e-01,  -6.49862352e-01],
        [  9.60667517e-01,   2.64129982e-01,   2.19728411e-01, ...,
           6.32828482e-01,   6.65661362e-01,   1.01745223e+00],
        [  5.28591092e-01,   4.36622885e-01,   4.92304307e-01, ...,
          -9.83703780e-02,   1.89448337e-01,   9.71220380e-02],
        ..., 
        [  8.72096861e-03,   2.35916860e-04,   1.44170673e-02, ...,
          -1.78555491e-03,   4.50170298e-03,  -1.80370267e-03],
        [ -6.86720290e-03,   2.25146981e-03,   5.80952570e-05, ...,
          -2.44652860e-03,   1.00039815e-02,   9.35630354e-04],
        [ -2.00927812e-02,   7.06540451e-03,  -1.09924506e-02, ...,
           2.83233085e-03,   1.67228555e-02,   4.09720668e-03]])

In [37]:
X_pred

array([[  3.36925323e+00,   1.80692294e+00,   9.26358664e-01, ...,
         -7.18297387e-03,   2.76884585e-02,   0.00000000e+00],
       [  1.57973152e+00,  -4.51489014e-02,   1.28513494e-01, ...,
          1.02554245e-02,   1.28414505e-03,   0.00000000e+00],
       [ -1.25916557e-01,  -7.64149756e-02,   7.14835915e-02, ...,
          1.79939758e-02,  -2.48033607e-03,   0.00000000e+00],
       ..., 
       [  1.54618006e+00,  -8.55687540e-02,   2.12877678e-01, ...,
         -1.66078411e-03,  -3.90398342e-03,   0.00000000e+00],
       [  1.40082389e+00,   5.54048392e-02,  -4.64107376e-01, ...,
          1.59499597e-02,  -2.49175881e-03,   0.00000000e+00],
       [  1.49485234e+00,   1.58047074e+00,   7.57591317e-01, ...,
         -6.20756724e-03,   1.93855187e-02,   0.00000000e+00]])

In [84]:
test_data_matrix

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [38]:
np.shape(X_pred)

(943, 1682)

In [42]:
np.shape(predcuda)

(943, 1682)

In [43]:
np.shape(test_data_matrix)

(943, 1682)

In [48]:
type(test_data_matrix)

numpy.ndarray

In [50]:
np.array(predcuda)

array([[  1.01566779e+00,   1.90207861e+00,   8.50892734e-01, ...,
          1.99125770e-02,   9.38555380e-01,  -6.49862352e-01],
       [  9.60667517e-01,   2.64129982e-01,   2.19728411e-01, ...,
          6.32828482e-01,   6.65661362e-01,   1.01745223e+00],
       [  5.28591092e-01,   4.36622885e-01,   4.92304307e-01, ...,
         -9.83703780e-02,   1.89448337e-01,   9.71220380e-02],
       ..., 
       [  8.72096861e-03,   2.35916860e-04,   1.44170673e-02, ...,
         -1.78555491e-03,   4.50170298e-03,  -1.80370267e-03],
       [ -6.86720290e-03,   2.25146981e-03,   5.80952570e-05, ...,
         -2.44652860e-03,   1.00039815e-02,   9.35630354e-04],
       [ -2.00927812e-02,   7.06540451e-03,  -1.09924506e-02, ...,
          2.83233085e-03,   1.67228555e-02,   4.09720668e-03]])

In [250]:
salx=train_data_matrix[:100,:100]
salxx=np.loadtxt('salx.txt')
wsalx=salx+1
wsalx

array([[ 6.,  4.,  5., ...,  5.,  4.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 5.,  1.,  4., ...,  6.,  1.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [251]:
np.savetxt('wsalx.txt', wsalx,'%.1f')

In [143]:
np.savetxt('test_data_matrix.txt', test_data_matrix)

In [275]:
tdm=train_data_matrix+1
np.savetxt('train_data_matrixT1.txt', tdm.transpose(),'%.1f')
#np.shape(tdm.transpose())

In [145]:
ux,sx,Vx = linalg.svd(salx)

In [228]:
uxx,sxx,Vxx = linalg.svd(salxx)
sxx

array([  9.58602932e+01,   3.25901370e+01,   3.13124506e+01,
         2.76886371e+01,   2.65456448e+01,   2.61468163e+01,
         2.56156840e+01,   2.38703685e+01,   2.36119872e+01,
         2.22185818e+01,   2.10597478e+01,   2.05656541e+01,
         2.01578602e+01,   1.96186762e+01,   1.90490209e+01,
         1.81962821e+01,   1.78093268e+01,   1.72146086e+01,
         1.64828623e+01,   1.61679754e+01,   1.59714308e+01,
         1.50321873e+01,   1.45306975e+01,   1.38365889e+01,
         1.35322029e+01,   1.31128377e+01,   1.29768691e+01,
         1.26217715e+01,   1.24316081e+01,   1.18319193e+01,
         1.13530634e+01,   1.10238288e+01,   1.07856987e+01,
         1.02609339e+01,   1.01351440e+01,   9.67962407e+00,
         9.27896451e+00,   9.12011414e+00,   8.86923109e+00,
         8.59454012e+00,   8.39833903e+00,   7.86917249e+00,
         7.67233771e+00,   7.33544241e+00,   6.85289590e+00,
         6.65129967e+00,   6.52440706e+00,   5.84668404e+00,
         5.73095171e+00,

In [146]:
sx

array([  9.58602932e+01,   3.25901370e+01,   3.13124506e+01,
         2.76886371e+01,   2.65456448e+01,   2.61468163e+01,
         2.56156840e+01,   2.38703685e+01,   2.36119872e+01,
         2.22185818e+01,   2.10597478e+01,   2.05656541e+01,
         2.01578602e+01,   1.96186762e+01,   1.90490209e+01,
         1.81962821e+01,   1.78093268e+01,   1.72146086e+01,
         1.64828623e+01,   1.61679754e+01,   1.59714308e+01,
         1.50321873e+01,   1.45306975e+01,   1.38365889e+01,
         1.35322029e+01,   1.31128377e+01,   1.29768691e+01,
         1.26217715e+01,   1.24316081e+01,   1.18319193e+01,
         1.13530634e+01,   1.10238288e+01,   1.07856987e+01,
         1.02609339e+01,   1.01351440e+01,   9.67962407e+00,
         9.27896451e+00,   9.12011414e+00,   8.86923109e+00,
         8.59454012e+00,   8.39833903e+00,   7.86917249e+00,
         7.67233771e+00,   7.33544241e+00,   6.85289590e+00,
         6.65129967e+00,   6.52440706e+00,   5.84668404e+00,
         5.73095171e+00,

In [147]:
sxx=np.loadtxt('Sx.txt')
Uxx= np.loadtxt('Ux.txt') 
vtxx=np.loadtxt('VTx.txt')


In [154]:
h=np.matrix(Uxx)*np.diag(sxx)*np.matrix(vtxx) 
h

matrix([[  5.92955886e-07,   1.47842232e-06,   1.35192435e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.00000092e+00,  -2.66771475e-08,   3.72160772e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.99999774e+00,   5.20597790e-07,   1.72293201e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  6.58288892e-07,   2.84153885e-06,   2.46829398e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [  3.53436696e-06,  -1.40632312e-06,   1.18258136e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -3.68878821e-07,   6.10616811e-07,  -1.40331573e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [191]:
salx

array([[ 5.,  3.,  4., ...,  4.,  3.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  3., ...,  5.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [192]:
wh=h[:100,:100]
wh-salx

matrix([[ -4.99999941e+00,  -2.99999852e+00,  -3.99999865e+00, ...,
          -4.00000000e+00,  -3.00000000e+00,  -5.00000000e+00],
        [  3.00000092e+00,  -2.66771475e-08,   3.72160772e-06, ...,
           0.00000000e+00,   0.00000000e+00,  -5.00000000e+00],
        [  3.99999774e+00,   5.20597790e-07,   1.72293201e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        ..., 
        [  3.99999754e+00,   3.99999923e+00,   4.44418655e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
        [ -1.00000324e+00,   1.02446644e-06,  -2.99999691e+00, ...,
          -5.00000000e+00,   0.00000000e+00,  -5.00000000e+00],
        [  4.99999124e+00,   1.12016473e-05,   5.79001670e-06, ...,
           0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [175]:
predT=np.matrix(U[:, :943]) * np.diag(ss[:943]) * np.matrix(VT[:943, :])
np.shape(predT)

(1682, 943)

In [174]:
predcuda

matrix([[ -4.73213449e-06,   3.99999942e+00,  -3.76372606e-07, ...,
           2.01609406e-06,   2.99999692e+00,   3.86525549e-06],
        [  4.30880615e-06,   3.73929502e-06,   6.33303780e-07, ...,
           3.00000350e+00,  -2.23761497e-06,   3.00000070e+00],
        [  4.97741820e-07,   1.99999683e+00,   2.99999630e+00, ...,
          -2.36025021e-06,  -2.05917268e-06,   8.18279388e-07],
        ..., 
        [ -3.30166696e-08,  -1.81072948e-07,   1.08337793e-07, ...,
           1.12993807e-07,   1.36154000e-07,   7.54345308e-08],
        [ -3.30081120e-07,  -8.71494916e-08,  -1.39958201e-08, ...,
          -5.20067975e-08,   1.31347319e-07,  -2.99210527e-07],
        [ -1.37426201e-07,  -9.52974408e-08,   3.37245598e-07, ...,
           1.78555349e-07,   1.09092591e-07,   1.94159299e-07]])

In [162]:
train_data_matrix

array([[ 5.,  3.,  4., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 5.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  5.,  0., ...,  0.,  0.,  0.]])

In [166]:
prt

array([[  5.00000000e+00,   3.00000000e+00,   4.00000000e+00, ...,
          2.48065457e-16,   4.64905892e-16,   0.00000000e+00],
       [ -1.30458144e-13,   2.04593287e-13,   1.17128529e-14, ...,
         -1.45716772e-16,  -5.55111512e-17,   0.00000000e+00],
       [  1.70488623e-14,  -4.98281971e-14,  -8.41410275e-14, ...,
          1.25767452e-16,  -1.63064007e-16,   0.00000000e+00],
       ..., 
       [  5.00000000e+00,  -4.87457297e-15,  -6.66133815e-16, ...,
         -1.90819582e-16,  -3.55618313e-16,   0.00000000e+00],
       [ -5.45570533e-15,  -3.19883009e-15,  -2.32973363e-15, ...,
         -6.93889390e-17,  -2.98372438e-16,   0.00000000e+00],
       [ -1.15706056e-14,   5.00000000e+00,  -1.06564063e-14, ...,
          3.10515502e-16,   4.57966998e-16,   0.00000000e+00]])

In [180]:
np.where(predcuda>=5)

(array([  0,   0,   0, ..., 899, 912, 913]),
 array([  41,   92,  188, ..., 1354,  983,  342]))

In [185]:
predcuda[0,188]

5.0000008004053837

In [187]:
np.where(train_data_matrix>=5)

(array([  0,   0,   0, ..., 942, 942, 942]),
 array([  0,   5,   8, ..., 671, 720, 942]))

In [194]:
sa=np.loadtxt('Sa.txt')
VTa=np.loadtxt('VTa.txt')
Ua=np.loadtxt('Ua .txt')

In [272]:
dwh=np.dot(Ua,np.dot(np.diag(sa),VTa)) 
dwh

array([[ -5.39996031e-06,   3.20630294e-06,   1.58343130e-15, ...,
         -9.22711753e-07,   4.00000037e+00,   0.00000000e+00],
       [  3.00000279e+00,   2.04285496e-06,  -1.14449390e-15, ...,
         -5.47782227e-07,  -7.12988204e-06,   0.00000000e+00],
       [  4.00000301e+00,   8.59798423e-07,   4.15258242e-16, ...,
         -6.27354700e-07,   3.00000090e+00,   0.00000000e+00],
       ..., 
       [  3.99998746e+00,   2.02268989e-06,  -7.95140011e-16, ...,
          1.78081257e-06,   4.99999496e+00,   0.00000000e+00],
       [  2.99999219e+00,   1.47936185e-07,   1.96233916e-15, ...,
         -1.10659607e-06,  -9.45107562e-06,   0.00000000e+00],
       [  4.99999375e+00,   4.99999769e+00,  -3.60014451e-16, ...,
         -2.50658986e-06,   5.00000040e+00,   0.00000000e+00]])

In [269]:
salx

array([[ 5.,  3.,  4., ...,  4.,  3.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 4.,  0.,  3., ...,  5.,  0.,  5.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [240]:
for i in range(100):
    for j in range(100):
        if dwh[i,j]<0.5:
            dwh[i,j]=0.0

In [243]:
print('User-based CF MSE: ' + str(rmse(np.array(dwh), salx)))

User-based CF MSE: 3.62801111119


In [276]:
A=np.loadtxt('A.txt')
uA,sA,VA = linalg.svd(A)

In [277]:
uA

array([[ 0.17529482,  0.75763765, -0.62869461],
       [-0.91080216,  0.36724149,  0.18860838],
       [ 0.37377956,  0.53955434,  0.75443354]])

In [278]:
sA

array([ 7.01532736,  1.1336587 ])

In [280]:
VA

array([[-0.57260167,  0.81983372],
       [ 0.81983372,  0.57260167]])

In [282]:
np.dot(uA[:,:2],np.dot(np.diag(sA),VA)) 

array([[ -1.03111494e-16,   1.50000000e+00],
       [  4.00000000e+00,  -5.00000000e+00],
       [ -1.00000000e+00,   2.50000000e+00]])

In [283]:
A

array([[ 0. ,  1.5],
       [ 4. , -5. ],
       [-1. ,  2.5]])

In [290]:
s32=np.loadtxt('./salida_A_3x2/output_S.txt')
VT32=np.loadtxt('./salida_A_3x2/output_VT.txt')
U32=np.loadtxt('./salida_A_3x2/output_U.txt')
A32=np.loadtxt('./salida_A_3x2/salida_A_3x2.txt') 
A32

ValueError: could not convert string to float: b'1.500000000,'

In [286]:
np.dot(U32[:,:2],np.dot(np.diag(s32),VT32))

array([[  1.49999891e+00,  -9.99999211e-01],
       [  4.00000218e+00,   2.49999704e+00],
       [ -4.99999666e+00,   1.59667653e-07]])

In [288]:
np.diag(s32)

array([[ 6.71982,  0.     ],
       [ 0.     ,  2.3117 ]])

In [289]:
VT32

array([[-0.975769, -0.218805],
       [ 0.218805, -0.975769]])

In [291]:
s100=np.loadtxt('./salida_wsalx_100x100/output_S.txt')
VT100=np.loadtxt('./salida_wsalx_100x100/output_VT.txt')
U100=np.loadtxt('./salida_wsalx_100x100/output_U.txt')

In [297]:
d100=np.dot(U100,np.dot(np.diag(s100),VT100))
d100

array([[  8.13762143e-07,   9.99989288e-01,   9.99995041e-01, ...,
          9.99991236e-01,   4.99997302e+00,   9.99995041e-01],
       [  4.00000096e+00,   9.99996232e-01,   9.99998845e-01, ...,
          9.99997147e-01,   9.99991294e-01,   9.99998845e-01],
       [  4.99999483e+00,   9.99996601e-01,   9.99997912e-01, ...,
          9.99998225e-01,   3.99999181e+00,   9.99997912e-01],
       ..., 
       [  5.00000604e+00,   9.99998316e-01,   9.99999109e-01, ...,
          9.99999345e-01,   5.99998977e+00,   9.99999109e-01],
       [  4.00000160e+00,   9.99995830e-01,   9.99998954e-01, ...,
          9.99998489e-01,   9.99987042e-01,   9.99998954e-01],
       [  5.99998121e+00,   5.99999238e+00,   9.99993722e-01, ...,
          9.99993304e-01,   5.99997500e+00,   9.99993722e-01]])

In [318]:
np.where((wsalx>1.99) & (wsalx<2.99))

(array([ 0,  0,  0,  0,  0,  0,  4,  4, 12, 12, 12, 12, 12, 12, 12, 12, 13,
        14, 19, 21, 42, 42, 48, 48, 48, 48, 48, 48, 48, 48, 48, 53, 55, 57,
        57, 61, 61, 63, 64, 64, 65, 71, 72, 81, 82, 89, 93, 94, 94, 95]),
 array([ 7, 20, 28, 34, 73, 77, 62, 68, 16, 23, 24, 36, 52, 66, 77, 85, 69,
         6, 68, 28, 46, 50,  1, 37, 39, 49, 67, 76, 79, 81, 89, 23, 87, 19,
        68, 32, 67, 31,  6, 69, 20,  6, 93, 20, 55, 68, 33,  2, 31, 41]))

In [319]:
np.where((d100>1.99) & (d100<2.99))

(array([ 1,  2,  6,  6,  6,  7, 16, 19, 20, 20, 20, 23, 23, 24, 28, 28, 31,
        31, 32, 33, 34, 36, 37, 39, 41, 46, 49, 50, 52, 55, 62, 66, 67, 67,
        68, 68, 68, 68, 69, 69, 73, 76, 77, 77, 79, 81, 85, 87, 89, 93]),
 array([48, 94, 14, 64, 71,  0, 12, 57,  0, 65, 81, 12, 53, 12,  0, 21, 63,
        94, 61, 93,  0, 12, 48, 48, 95, 42, 48, 42, 12, 82,  4, 12, 48, 61,
         4, 19, 57, 89, 13, 64,  0, 48,  0, 12, 48, 48, 12, 55, 48, 72]))

In [320]:
wsalx

array([[ 6.,  4.,  5., ...,  5.,  4.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 5.,  1.,  4., ...,  6.,  1.,  6.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.]])

In [321]:
sf=np.loadtxt('./salida_train_1682x943/output_S.txt')
VTf=np.loadtxt('./salida_train_1682x943/output_VT.txt')
Uf=np.loadtxt('./salida_train_1682x943/output_U.txt')

In [323]:
df=np.dot(Uf[:,:943],np.dot(np.diag(sf),VTf))
df

array([[  8.12405111e-06,   1.00000774e+00,   1.00000437e+00, ...,
          1.00000280e+00,   1.00000420e+00,   1.00000357e+00],
       [  1.00000446e+00,   1.00000585e+00,   3.00000340e+00, ...,
          1.00000084e+00,   1.00000230e+00,   1.00000176e+00],
       [  1.00000544e+00,   1.00000322e+00,   4.00000347e+00, ...,
          1.00000060e+00,   1.00000191e+00,   1.00000116e+00],
       ..., 
       [  9.99999562e-01,   4.00000342e+00,   1.00000481e+00, ...,
          1.00000125e+00,   1.00000285e+00,   1.00000224e+00],
       [  4.00000742e+00,   1.00000240e+00,   1.00000387e+00, ...,
          1.00000166e+00,   1.00000312e+00,   1.00000264e+00],
       [  9.99997363e-01,   4.00000320e+00,   1.00000669e+00, ...,
          9.99999899e-01,   1.00000139e+00,   1.00000070e+00]])

In [329]:
tdm

array([[ 6.,  4.,  5., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       ..., 
       [ 6.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  1.,  1., ...,  1.,  1.,  1.],
       [ 1.,  6.,  1., ...,  1.,  1.,  1.]])

In [332]:
np.sum(np.where((df>0.99) & (df<1.99)))

1993654169

In [333]:
np.sum(np.where((tdm>0.99) & (tdm<1.99)))

2009352508